# 第六节 实战召回优化——相似度优化

上一节我们讲了节点组的优化技巧，这一节内容我们将在代码层面实现优化召回率的其中一种方式——相似度优化。

## 提升指标的优化策略之相似度优化

**检索方式的选择**其实挺有意思的。我们团队试过各种组合，最后发现没有银弹。向量检索对语义理解好，BM25 对关键词匹配准，两个结合起来用效果最好。有个小技巧——设置相似度阈值的时候别太激进，我一般设在 0.7 左右，太高了容易漏召回，太低了又全是垃圾。

那么如何从**代码层面实现相似度优化策略**呢？

## 相似度优化

LazyLLM 原生支持 cosine 相似度和 bm25 相似度，其中 cosine 检索需要对文档进行 Embedding 后才能进行计算。

Embedding 这东西说白了就是**把文本变成一串数字——准确说是高维向量**。刚接触的时候我也懵，后来想明白了，就像给**每段文字贴个坐标**，**相似的内容坐标就近**。

LazyLLM 在这方面做得还挺贴心，支持**在线和离线**两种模式。

在线模型用起来简单，`OnlineEmbeddingModule`搞定一切。支持的平台还挺多——OpenAI、商汤、智谱、通义千问、豆包都有。这里给出通义千问的例子：

In [ ]:
pip install lazyllm
export QWEN_API_KEY='sk-这里填写你的key'

In [ ]:
from lazyllm import OnlineEmbeddingModule
import os


online_embed = OnlineEmbeddingModule(
    source="qwen",
    api_key=os.getenv("QWEN_API_KEY"),
)


print("online embed: ", online_embed("hello world"))	#这里输出字符串hello world的高维向量

有个经验想分享：如果你们公司自己部署了 Embedding 服务，接口格式又跟 LazyLLM 支持的不一样，可以继承`OnlineEmbeddingModuleBase`自己写个适配器。我之前就这么干过，主要是重写`_encapsulated_data`和`_parse_response`两个方法，把数据格式转来转去。

最后说个省事儿的技巧——用 LazyLLM 处理文档时，不用自己循环调用 Embedding。直接在创建`Document`时传入`embed`参数就行：

```python
# 加载文档
documents = lazyllm.Document('./data_kb', embed=online_embed)
```

这里还有个细节很多人不知道：LazyLLM 很聪明，它不**会立即对所有文档做嵌入**，而是**等到第一次检索时才根据你选的相似度计算方式决定要不要嵌入**。如果你用 BM25 这种基于词频的方法，它就不会浪费时间做向量化。

用了这么久，我的感受是：选 Embedding 模型别太纠结，主流的几个差别不大。关键是要根据业务场景调优——中文用 bge 系列，英文用 e5 系列，多语言就上 m3。预算充足就用在线 API，追求性价比就本地部署。真正影响效果的往往不是模型本身，而是你的数据质量和检索策略。

### 使用 embedding 召回文档

学会指定嵌入模型后，我们就可以通过向量相似度进行文档召回了。LazyLLM 原生支持 cosine 相似度进行语义检索。为文档指定 Embedding 后，`Retriever` 传入参数 `similarity=“cosine”` 即可。`LazyLLM.Retriever` 可传入的值有：“cosine”，“bm25”和“bm25_chinese”，其中只有“cosine”需要为文档指定 Embedding。需要注意的是，哪怕为文档指定了 Embedding，但检索时使用 “bm25” 和 “bm25_chinese” 相似度时并不会进行嵌入计算。下面这段代码展示了使用原生相似度进行召回的例子，并简单对比了 cosine 和 bm25 的的召回特点：

In [ ]:
from lazyllm import OnlineEmbeddingModule, Document, Retriever
import os


online_embed = OnlineEmbeddingModule(
    source="qwen",
    api_key=os.getenv("QWEN_API_KEY"),
)


# 文档加载
docs = Document("./data_kb", embed=online_embed)
docs.create_node_group(name="block", transform=(lambda d: d.split("\n")))

# 定义两个不同的检索器在同一个节点组使用不同相似度方法进行检索
retriever1 = Retriever(docs, group_name="block", similarity="cosine", topk=3)
retriever2 = Retriever(docs, group_name="block", similarity="bm25_chinese", topk=3)

# 执行查询
query = "都有谁参加了2008年的奥运会？"
result1 = retriever1(query=query)
result2 = retriever2(query=query)

print("余弦相似度召回结果：")
print("\n\n".join([res.get_content() for res in result1]))
print("bm25召回结果：")
print("\n\n".join([res.get_content() for res in result2]))

下面是二者的召回结果，可见使用不同相似度对召回结果有一定的影响，从查询特点来看，包含“2008 年”和“奥运会”两个比较重要的主题，余弦相似度的检索结果同时包含了这两个重要主题。但 bm25 的结果虽然每条结果都包含“奥运会”的信息，但其中有一个结果并没有提及 2008 年奥运会。这是因为 bm25 方法在分词后将 2008 年的奥运会分为了两个 token，在计算时并没有将其视作一个整体的语义信息，所以会有尽管关键词充分，但没有切住主题的问题。

|     | **余弦相似度召回结果**                                                                                | **bm25 召回结果**                                                                                                                                                                                                |
| --- | ----------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1   | ...参加 2008 年北京夏季奥林匹克运动会的中国花样游泳队一行共计 15 人，其中运动员 9 人，教练 4 人...    | ...1996 年和 2000 年夏季奥林匹克运动会两度获得举重项目金牌，为中国奥运史上连续两届获得举重冠军的第一人...参加了亚特兰大奥运会的 70 公斤级举重比赛...成为该届奥运会的经典一幕。悉尼奥运会后...第三次参加奥运会... |
| 2   | 2008 年夏季奥林匹克运动会男子 100 米跑赛事在 2008 年 8 月 15 日及 8 月 16 日在北京国家体育馆举行...   | 2008 年 8 月 10 日，谭望嵩在北京奥运会中国队与比利时队的男足小组赛当中...                                                                                                                                        |
| 3   | 参加 2008 年北京夏季奥林匹克运动会的中国皮划艇队共计 40 人，其中运动员 23 人，官员与工作人员 17 人... | ...参加 2008 年北京夏季奥林匹克运动会的中国花样游泳队一行共计 15 人，其中运动员 9 人，教练 4 人...                                                                                                               |

### 多 embedding 召回

那么能不能**针对同一个知识库应用多个 embedding 呢**？也是可以的，这里给出参考：

```python
from lazyllm import OnlineEmbeddingModule, TrainableModule, Document

online_embed = OnlineEmbeddingModule()
offline_embed = TrainableModule('bge-large-zh-v1.5').start()
embeds = {'vec1': online_embed, 'vec2': offline_embed}
doc = Document('./data_kb', embed=embeds)
```

这段代码与使用单个 embedding 相比，只是将多个模型构造成一个字典传入了。

### 相似度阈值过滤

你可能会问了，**我能不能过滤掉低相似度的结果，只使用高相似度的结果呢**？

当然是可以的。通过阈值过滤的方法可以过滤掉相似度过低的节点，提升召回文档的可靠性。假设在没有使用阈值过滤时系统在 `topk=6` 的时候召回了 6 篇文档，其中相似度依次为 `0.9, 0.87, 0.6, 0.31, 0.23, 0.21`，通过观察节点内容我们发现后面的三个文档与查询并没有直接的关系时，我们可以通过设定过滤的阈值为 0.6，此时召回器仅返回相似度大于等于 0.6 的节点。LazyLM 中设定召回器相似度阈值的方法是传入 `similarity_cut_off`：

In [ ]:
from lazyllm import Document, Retriever, OnlineEmbeddingModule
import os

# 定义 embedding 模型
online_embed = OnlineEmbeddingModule(
    source="qwen",
    api_key=os.getenv("QWEN_API_KEY"),
)


# 文档加载
docs = Document("./data_kb", embed=online_embed)
docs.create_node_group(name="block", transform=(lambda d: d.split("\n")))

# 定义两个不同的检索器对比是否使用阈值过滤的效果
retriever1 = Retriever(docs, group_name="block", similarity="cosine", topk=6)
retriever2 = Retriever(
    docs, group_name="block", similarity="cosine", similarity_cut_off=0.6, topk=6
)

# 执行检索
query = "2008年参加花样游泳的运动员都有谁？"
result1 = retriever1(query=query)
result2 = retriever2(query=query)

print("未设定 similarity_cut_off：")
print("\n\n".join([res.get_content() for res in result1]))
print("设定 similarity_cut_off=0.6 ：")
print("\n\n".join([res.get_content() for res in result2]))

通过比较上述视频中使用相似度过滤方法与否的结果，可以发现使用相似度过滤有效减少了召回结果中与查询相关度较低的冗余上下文，有助于后续流程中的生成效果提升。